## Physics-Informed Machine Learning (PIML) и Scientific Machine Learning (SciML) 

Классический подход решения математических задач широко известен и изучен, он включает решение уравнений численными методами (FEM, FDM, FVM), требует сеточной дискретизации, может быть ресурсоёмким.

Как было ранее выяснено, модель машинного обучения $f_\theta(x)$ обучается на данных $\{(x_i, y_i)\}$, то есть пусть они будут размеченными, однако без учёта физики - модель не имеет представлений о мире и никак не "предрасположена" к пониманию фундаментальных заново физики, а значит может их нарушать. Однако важным свойством является то, что модель хорошо аппроксимирует распределение данных.

**Physics-Informed Machine Learning (PIML)**, или **Scientific Machine Learning (SciML)**, объединяет методы машинного обучения с физическими, химическими и инженерными моделями. Цель — получать гибридные модели, которые одновременно:
1. Используют **вычислительные преимущества** нейронных сетей и статистических методов.
2. Интегрируют **известные физические законы**, отсюда и области применения.

### Physics-Informed подход

Обучение модели $u_\theta(\mathbf{x})$ (например, нейронной сети) с учётом физики через доп. слагаемое в функции потерь.

Пусть мы хотим решить уравнение в частных производных (PDE):

$$
\mathcal{N}[u(\mathbf{x})] = 0, \quad \mathbf{x}\in\Omega,\quad
\begin{cases}
u(\mathbf{x}) = g(\mathbf{x}), & \mathbf{x}\in\partial\Omega_D,\\
\mathcal{B}[u] = h(\mathbf{x}), & \mathbf{x}\in\partial\Omega_N.
\end{cases}
$$

Обучаем нейронную сеть $u_\theta(\mathbf{x})$, минимизируя комбинированную функцию потерь:

$$
\mathcal{L}(\theta)
= \underbrace{\frac1{N_u}\sum_{i=1}^{N_u} \bigl|u_\theta(\mathbf{x}_i) - y_i \bigr|^2}_{\displaystyle\text{Data Loss}}
+ \underbrace{\frac\lambda{N_r}\sum_{j=1}^{N_r} \bigl|\mathcal{N}[u_\theta](\mathbf{x}_j)\bigr|^2}_{\displaystyle\text{Physics Loss}}
+ \underbrace{\frac\mu{N_b}\sum_{k=1}^{N_b} \bigl|\mathcal{B}[u_\theta](\mathbf{x}_k) - h(\mathbf{x}_k)\bigr|^2}_{\displaystyle\text{Boundary Loss}},
$$

где:

* $\{\mathbf{x}_i,y_i\}$ — обучающая выборка (если есть наблюдения);
* $\{\mathbf{x}_j\}$ — точки, где проверяется PDE;
* $\{\mathbf{x}_k\}$ — граничные точки для граничных условий;
* $\lambda,\mu$ — весовые коэффициенты.

### Универсальная теорема аппроксимации

[Цыбенко, 1989](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%A6%D1%8B%D0%B1%D0%B5%D0%BD%D0%BA%D0%BE)

Пусть $ \sigma\colon\mathbb{R}\to\mathbb{R} $ - непрерывная сигмоидная функция (то есть $ \lim_{t\to-\infty}\sigma(t)=0 $, $ \lim_{t\to+\infty}\sigma(t)=1 $, например $ 1/(1+e^t) $). 
Тогда для любой непрерывной функции $ f\in C([0,1]^d) $ и любого $ \varepsilon>0 $ существует параметризованная функция вида:  
$$
F(\mathbf{x})
=\sum_{j=1}^N \alpha_j\,\sigma(\mathbf{w}_j^\top \mathbf{x} + b_j),
$$
где $N\in\mathbb{N}$, $\alpha_j\in\mathbb{R}$, $\mathbf{w}_j\in\mathbb{R}^d$, $b_j\in\mathbb{R}$, такая что  
$$
\sup_{\mathbf{x}\in[0,1]^d}\bigl|F(\mathbf{x}) - f(\mathbf{x})\bigr| < \varepsilon.
$$

> Теорема утверждает, что искусственная нейронная сеть с одним скрытым слоем может аппроксимировать любую непрерывную функцию многих переменных с любой наперед заданной точностью. Условиями являются достаточное количество нейронов скрытого слоя, "удачный" подбор параметров этого слоя. 

### Теорема суперпозиции Колмогорова–Арнольда

[Колмогоров, Арнольд, 1957](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%9A%D0%BE%D0%BB%D0%BC%D0%BE%D0%B3%D0%BE%D1%80%D0%BE%D0%B2%D0%B0_%E2%80%94_%D0%90%D1%80%D0%BD%D0%BE%D0%BB%D1%8C%D0%B4%D0%B0)

Любая непрерывная функция $f\colon[0,1]^d\to\mathbb{R}$ может быть представлена в виде  
$$
f(\mathbf{x})
=\sum_{q=0}^{2d}\Phi_q\!\Bigl(\sum_{p=1}^d\psi_{p,q}(x_p)\Bigr),
$$
где $\{\psi_{p,q}\}$ и $\{\Phi_q\}$ - непрерывные функции одной переменной.

> Теорема утверждает, что каждая многомерная непрерывная функция может быть представлена в виде суперпозиции непрерывных функций одной переменной. 

### Характеризация функций активации

[Пинкуc, 1999](https://pinkus.net.technion.ac.il/files/2021/02/acta.pdf)

Пусть $K\subset\mathbb{R}^d$ - компакт, и рассмотрим класс функций

$$
\mathcal{H}_\sigma \;=\;
\Bigl\{\,h(\mathbf{x}) = \sum_{j=1}^N \alpha_j\,\sigma(\mathbf{w}_j^\top\mathbf{x} + b_j)\;\Big|\; 
N\in\mathbb{N},\;\alpha_j\in\mathbb{R},\;\mathbf{w}_j\in\mathbb{R}^d,\;b_j\in\mathbb{R}
\Bigr\}.
$$

Обозначим через $C(K)$ пространство всех непрерывных функций на $K$ с нормой $\|f\|_\infty = \sup_{\mathbf{x}\in K}|f(\mathbf{x})|$.

Пусть $\sigma\colon\mathbb{R}\to\mathbb{R}$ - непрерывная функция. Тогда следующие утверждения эквивалентны:
1. $\sigma$ **не совпадает ни с каким полиномом** на каком-либо открытом интервале $\mathcal{I}\subset\mathbb{R}$.
2. Класс $\mathcal{H}_\sigma$ **плотен** в $C(K)$ для любого компакта $K\subset\mathbb{R}^d$; то есть для любой $f\in C(K)$ и любого $\varepsilon>0$ существует $h\in\mathcal{H}_\sigma$ с
$$
\|f - h\|_\infty < \varepsilon.
$$


> Если $\sigma$ на некотором интервале точно совпадает с полиномом степени $m$, то все выражения $ \sigma(\mathbf{w}^\top\mathbf{x}+b) $ являются полиномиальными функциями переменной $ \mathbf{w}^\top\mathbf{x}+b $. Линейные комбинации таких полиномов не могут аппроксимировать, скажем, непрерывные функции, не лежащие в пространстве полиномов степени $m$.

> Если же $\sigma$ нигде не полиномиальная, то с помощью классических конструкций теории аппроксимации можно показать, что $\mathcal{H}_\sigma$ содержит достаточно функций, чтобы покрыть всё $C(K)$.

Кроме того:

- [Теорема Вейерштрасса-Стоуна](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%92%D0%B5%D0%B9%D0%B5%D1%80%D1%88%D1%82%D1%80%D0%B0%D1%81%D1%81%D0%B0_%E2%80%94_%D0%A1%D1%82%D0%BE%D1%83%D0%BD%D0%B0)
- [Теормеа Хорника](https://www.cs.cmu.edu/~epxing/Class/10715/reading/Kornick_et_al.pdf)
- [Теорема Яроцкого](https://arxiv.org/abs/1610.01145)

### Теорема сходимости PINN

[Mishra & Molinaro, 2022](https://arxiv.org/abs/2006.16144)

Рассмотрим PINN $u_\theta$, обучаемую для решения корректной задачи, при наличии достаточного числа точек коллокации и нейронная сеть удовлетворяет условиям универсальной теоремы аппроксимации. При регулярности истинного решения $u$ существует набор весов $\theta$, при котором общий для функции потерь справедливо:  
$$
\mathcal{L}(\theta)
=\frac1{N_u}\sum_i|u_\theta(x_i)-u(x_i)|^2
+\frac{\lambda}{N_r}\sum_j|\mathcal{N}[u_\theta](x_j)|^2
$$
стремится к нулю при $N_r\to\infty$, и при этом $\|u_\theta - u\|_{L^2}$ можно связать с величиной $\mathcal{L}(\theta)$ и размером сети.

> То есть, при достаточной обобщающей и аппроксимирующей возможностях сети (архитектура), малости потерь (оптимизатор) и корректной постановке задачи (само решение), PINN сходится к решению дифференциального уравнения.

Важно отметить, что теорема, помимо прочего, вводит источники ошибок полученной аппроксимации конкретного решения.

<img src="../images/errors.jpg" alt="Компоненты ошибок" width="800">

##### Основные модели
- **Physics-Informed Neural Networks (PINNs)**
- **Neural Operators**
- **DeepONet**

##### Примеры

- **Ускорение численных симуляций:** замена тяжелых итераций ML-аппроксимацией локальных шагов решения.
- **Нейронные операторы и генерализация:** обучение сразу на целый класс задач.
- **Неопределённость и верификация:** учет неопределенности данных и физической модели.
- **Мультиизмерные и многофизичные задачи:** совмещение различных областей физики.
- **Интеграция с экспериментальными данными:** адаптивное обучение при поступлении новых измерений.

##### Области применения

- **Гидродинамика и аэродинамика:** моделирование течений, турбулентности, оптимизация крыльев.
- **Метеорология и климатология:** прогнозирование погоды, оценка риска экстремальных явлений.
- **Биомедицинское моделирование:** распространение заболеваний, динамика лекарств, биомеханика тканей.
- **Материаловедение:** предсказание свойств новых сплавов и композитов, фазовые переходы.
- **Электроника и энергетика:** оптимизация ячеек топливных элементов, управление сетью.

> Подробнее можно о конкретных примера можно узнать в [видео](https://www.youtube.com/watch?v=O09lu-lsLhU&ab_channel=MSU_AI).

### Решение ОДУ с помощью нейронной сети

Рассмотрим простейшее ОДУ первого порядка с начальным условием:
$$
\begin{cases}
u'(x) + u(x) = 0, & x \in [0, X_{\max}],\\
u(0) = 1.
\end{cases}
$$

Аналитическое решение  
$$
u(x) = e^{-x}.
$$

Общая функция потерь состоит из двух слагаемых:
1. **Physics loss** — MSE по коллокационным точкам \(\{x_i\}\):
$$
\mathcal{L}_{\mathrm{phys}}(\theta)
= \frac{1}{N_f} \sum_{i=1}^{N_f} \bigl(u_\theta'(x_i) + u_\theta(x_i)\bigr)^2.
$$
2. **Initial condition loss** — MSE по начальному условию:
$$
\mathcal{L}_{\mathrm{IC}}(\theta)
= \bigl(u_\theta(0) - 1\bigr)^2.
$$

Итоговая функция:
$$
\mathcal{L}(\theta)
= \mathcal{L}_{\mathrm{phys}}(\theta)
+ \lambda \,\mathcal{L}_{\mathrm{IC}}(\theta),
$$
где $\lambda$ — взвешивающий коэффициент (например, $\lambda=1$).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.integrate import solve_ivp
from tqdm.notebook import tqdm

In [ ]:
class PINN(nn.Module):
    """
    Физически информированная нейронная сеть для приближенного решения ОДУ.
    """
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden1 = nn.Linear(1, 20)
        self.hidden2 = nn.Linear(20, 20)
        self.hidden3 = nn.Linear(20, 20)
        self.out = nn.Linear(20, 1)
        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.activation(self.hidden1(x))
        x = self.activation(self.hidden2(x))
        x = self.activation(self.hidden3(x))
        return self.out(x)

In [ ]:
def compute_loss(model, x_colloc, x_ic):
    """
    Вычисляет функцию потерь для PINN.

    Аргументы:
        model (nn.Module): обучаемая модель PINN.
        x_colloc (torch.Tensor): тензор коллокационных точек.
        x_ic (torch.Tensor): тензор для начального условия.

    Возвращает:
        loss (torch.Tensor): суммарное значение функции потерь.
    """
    x_colloc.requires_grad = True
    u_pred = model(x_colloc)

    grad_outputs = torch.ones_like(u_pred)
    du_dx = torch.autograd.grad(
        u_pred, x_colloc,
        grad_outputs=grad_outputs,
        create_graph=True
    )[0]

    # f(x) = u'(x) + u(x)
    f = du_dx + u_pred
    mse_pde = torch.mean(f ** 2)

    # u(0) = 1
    u_ic_pred = model(x_ic)
    mse_ic = torch.mean((u_ic_pred - 1.0) ** 2)

    loss = mse_pde + mse_ic
    return loss

In [ ]:
def train(model: nn, 
          optimizer: optim, 
          num_epochs: int, 
          x_colloc: torch.Tensor, 
          x_ic: torch.Tensor):
    """
    Обучает модель PINN.

    Аргументы:
        model (nn.Module): обучаемая модель PINN.
        optimizer (torch.optim.Optimizer): оптимизатор.
        num_epochs (int): количество эпох обучения.
        x_colloc (torch.Tensor): коллокационные точки.
        x_ic (torch.Tensor): точка начального условия.
    """
    for epoch in tqdm(range(num_epochs), desc="Training epochs"):
        optimizer.zero_grad()
        loss = compute_loss(model, x_colloc, x_ic)
        loss.backward()
        optimizer.step()

        if epoch % 1000 == 0:
            print(f"Epoch {epoch:5d}, Loss: {loss.item():.6f}")

In [ ]:
model = PINN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5000
batch_size = 100

x_colloc = torch.linspace(0, 1, batch_size).view(-1, 1)
x_ic = torch.tensor([[0.0]])

In [ ]:
train(model, optimizer, num_epochs, x_colloc, x_ic)

In [ ]:
x_test = torch.linspace(0, 1, 100).view(-1, 1)

with torch.no_grad():
    u_pred = model(x_test).cpu().numpy().flatten()

In [ ]:
x_test_np = x_test.cpu().numpy().flatten()
u_true = np.exp(-x_test_np)

sol = solve_ivp(lambda t, u: -u, [0, 1], [1.0], t_eval=x_test_np)
u_numerical = sol.y[0]

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(x_test_np, u_true, 'b-', label='Analytic')
plt.plot(x_test_np, u_pred, 'r--', label='PINN')
plt.plot(x_test_np, u_numerical, 'g-.', label='SciPy')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title("u'(x) + u(x) = 0")
plt.legend()
plt.grid(True)
plt.show()

### Обучение PINN для решения уравнения теплопроводности

Решим задачу одномерной теплопроводности с граничными условиями вида первого рода. Рассматриваем линейное уравнение теплопроводности и постановку задачи в целом:
$$
\begin{cases}
u_t(x,t) - \alpha\,u_{xx}(x,t) = 0, & (x,t) \in (0,1)\times(0,T],\\
u(x,0) = \sin(\pi x), & x \in [0,1],\\
u(0,t) = u(1,t) = 0, & t \in [0,T],
\end{cases}
$$

где $\alpha = 0.1$, $T = 2$.

Аналитическое решение этой задачи:
$$
u(x,t) = \sin(\pi x)\,\exp\bigl(-\alpha\,\pi^2\,t\bigr).
$$

Функция потерь для такой задачи будет иметь одно дополнительное слагаемое:
$$
\mathcal{L}(\theta)
= \underbrace{\frac{1}{N_f}\sum_{i=1}^{N_f} \bigl(u_{\theta,t}
- \alpha\,u_{\theta,xx}\bigr)^2}_{\displaystyle\text{Physics loss}}
+ \underbrace{\frac{1}{N_b}\sum_{j=1}^{N_b} \bigl(u_\theta(x_j, t_j)\bigr)^2}_{\displaystyle\text{Boundary loss}}
+ \underbrace{\frac{1}{N_0}\sum_{k=1}^{N_0} \bigl(u_\theta(x_k,0) - \sin(\pi x_k)\bigr)^2}_{\displaystyle\text{Initial condition loss}}.
$$

Для примера воспользуется библиотекой [DeepXDE](https://deepxde.readthedocs.io/en/latest/), одной из [первых](https://arxiv.org/abs/1907.04502) библиотек поддерживающих работу с PINN.

In [11]:
!pip install deepxde -q

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def heat_eq(x, u):
    du_t = dde.grad.jacobian(u, x, i=0, j=1) # du/dt
    du_xx = dde.grad.hessian(u, x, i=0, j=0) # du/dx
    return du_t - 0.1 * du_xx # α = 0.1

In [ ]:
# x ∈ [0, 1], t ∈ [0, 2]
geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

In [ ]:
# Boundary condition: u = 0 на границе области (при любом t)
bc = dde.icbc.DirichletBC(geomtime, 
                          lambda x: 0, lambda _, on_boundary: on_boundary)

In [ ]:
# Initial condition: u(x,0) = sin(πx)
ic = dde.icbc.IC(
    geomtime,
    lambda x: np.sin(np.pi * x[:, 0:1]),
    lambda _, on_initial: on_initial,
)

In [ ]:
# Keras style
net = dde.nn.FNN([2, 20, 20, 1], "tanh", "Glorot normal")
data = dde.data.TimePDE(geomtime, heat_eq, [bc, ic], 
                        num_domain=1000, 
                        num_boundary=100, 
                        num_initial=100, 
                        num_test=1000)
model = dde.Model(data, net)

model.compile("adam", lr=0.001)
losshistory, train_state = model.train(epochs=5000)

In [ ]:
x = np.linspace(0, 1, 101)
t = np.linspace(0, 2, 101)
X, T = np.meshgrid(x, t)
XT = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

u_pred = model.predict(XT)
u_pred = u_pred.reshape(X.shape)

u_exact = np.sin(np.pi * X) * np.exp(-0.1 * (np.pi**2) * T)

error = np.abs(u_exact - u_pred)

In [ ]:
fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(1, 3, 1)
contour1 = ax1.contourf(X, T, u_pred, 100, cmap="viridis")
ax1.set_title("PINN")
ax1.set_xlabel("x")
ax1.set_ylabel("t")
fig.colorbar(contour1, ax=ax1)

ax2 = fig.add_subplot(1, 3, 2)
contour2 = ax2.contourf(X, T, u_exact, 100, cmap="viridis")
ax2.set_title("Analytic")
ax2.set_xlabel("x")
ax2.set_ylabel("t")
fig.colorbar(contour2, ax=ax2)

ax3 = fig.add_subplot(1, 3, 3)
contour3 = ax3.contourf(X, T, error, 100, cmap="hot")
ax3.set_title("Absolute error")
ax3.set_xlabel("x")
ax3.set_ylabel("t")
fig.colorbar(contour3, ax=ax3)

plt.suptitle("PINN", 
             fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()